In [2]:
import random, sys, os
sys.path.insert(0, os.path.abspath('..'))
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from pkldataset import PKLDataset

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark   = False

class CNN(nn.Module):
    def __init__(self, input_length: int = 2800, num_classes: int = 10, input_channels: int = 1):
        super().__init__()
        # First convolutional block
        self.conv1 = nn.Sequential(
            nn.Conv1d(input_channels, 16, kernel_size=31, padding=15),
            nn.BatchNorm1d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2)
        )
        # Second convolutional block
        self.conv2 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=31, padding=15),
            nn.BatchNorm1d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2)
        )
        # Third convolutional block
        self.conv3 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=31, padding=15),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2)
        )
        
        # After three blocks with pooling (each reducing the length by a factor of 2),
        # the sequence length becomes input_length // 8.
        conv_output_length = input_length // 8
        
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * conv_output_length, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Ensure the input has shape [batch_size, 1, 2800]
        if x.dim() == 2:
            x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.fc_layers(x)
        return x

def train_model(model, train_loader, criterion, optimizer, scheduler,
                num_epochs, device, max_grad_norm=1.0):
    best_loss        = float('inf')
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            targets = targets.argmax(dim=1)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss    = criterion(outputs, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        scheduler.step()
        epoch_loss = running_loss / len(train_loader.dataset)
        if epoch_loss < best_loss:
            best_loss        = epoch_loss
            best_model_state = model.state_dict()

    # return the best‐seen parameters
    model.load_state_dict(best_model_state)
    return model

def eval_model(model, val_loader, device):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            targets = targets.argmax(dim=1)
            outputs = model(inputs)
            _, pred = outputs.max(1)
            total   += targets.size(0)
            correct += (pred == targets).sum().item()
    return 100. * correct / total

# === CONFIGURATION ===
train_path_1     = r"C:\Users\gus07\Desktop\data hiwi\preprocessing\HC\T197\RP"# add path to pretrain folder here (e.g. "RPHC197")
# Training dataset names
transfer_sets = ["../datasets/RPDC197/train_20", "../datasets/RPDC197/train_50", "../datasets/RPDC197/train_100", "../datasets/RPDC197/train_200", "../datasets/RPDC197/train_300",
 "../datasets/RPDC197/train_400", "../datasets/RPDC197/train_500", "../datasets/RPDC197/train_600"]

# Validation datasets to test each model on
val_paths = [
    "../datasets/RPDC185/val_1000",
    "../datasets/RPDC188/val_1000",
    "../datasets/RPDC191/val_1000",
    "../datasets/RPDC194/val_1000",
    "../datasets/RPDC197/val_1000",
]
seeds            = [101,202,303,404,505,606,707,808,909,1001]
device           = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion        = nn.CrossEntropyLoss()

# Prepare results container
# results[train_t][val_path] = list of accuracies (one per seed)
results = {
    t: {vp: [] for vp in val_paths}
    for t in transfer_sets
}

for seed in seeds:
    print(f"\n>>> Full pipeline with seed {seed}")
    set_seed(seed)

    # --- FIRST PHASE on train_path_1 ---
    train_ds, val_ds = PKLDataset.split_dataset(train_path_1)
    train_loader1 = DataLoader(train_ds, batch_size=64, shuffle=True)
    val_loader1   = DataLoader(val_ds,   batch_size=64, shuffle=True)

    model = CNN().to(device)
    opt   = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    sch   = optim.lr_scheduler.StepLR(opt, step_size=50, gamma=0.1)

    model = train_model(
        model, train_loader1,
        criterion, opt, sch,
        num_epochs=10,
        device=device
    )
    # keep pretrained weights in memory
    pretrained_state = model.state_dict()

    # --- SECOND PHASE (TRANSFER) ---
    for t in transfer_sets:
        # reload pretrained
        tl_model = CNN().to(device)
        tl_model.load_state_dict(pretrained_state)

        # train on t
        loader_t = DataLoader(PKLDataset(t), batch_size=64, shuffle=True)
        opt2     = optim.Adam(tl_model.parameters(), lr=1e-3, weight_decay=1e-5)
        sch2     = optim.lr_scheduler.StepLR(opt2, step_size=25, gamma=0.1)

        tl_model = train_model(
            tl_model, loader_t,
            criterion, opt2, sch2,
            num_epochs=100,
            device=device
        )

        # evaluate on each val_path
        for vp in val_paths:
            loader_vp = DataLoader(PKLDataset(vp), batch_size=64, shuffle=False)
            acc       = eval_model(tl_model, loader_vp, device)
            results[t][vp].append(acc)
            print(f"  [Seed {seed}] {t} → {vp}: {acc:.2f}%")

# --- FINAL SUMMARY ---
print("\n=== Mean ± Std Dev over seeds ===")
for t in transfer_sets:
    for vp in val_paths:
        acc_list = results[t][vp]
        mean_acc = np.mean(acc_list)
        std_acc  = np.std(acc_list, ddof=1)
        print(f"{t} → {vp}: mean = {mean_acc:.2f}%,  std = {std_acc:.2f}%")



>>> Full pipeline with seed 101
  [Seed 101] ../datasets/RPDC197/train_20 → ../datasets/RPDC185/val_1000: 57.79%
  [Seed 101] ../datasets/RPDC197/train_20 → ../datasets/RPDC188/val_1000: 60.29%
  [Seed 101] ../datasets/RPDC197/train_20 → ../datasets/RPDC191/val_1000: 81.21%
  [Seed 101] ../datasets/RPDC197/train_20 → ../datasets/RPDC194/val_1000: 85.70%
  [Seed 101] ../datasets/RPDC197/train_20 → ../datasets/RPDC197/val_1000: 95.30%
  [Seed 101] ../datasets/RPDC197/train_50 → ../datasets/RPDC185/val_1000: 63.03%
  [Seed 101] ../datasets/RPDC197/train_50 → ../datasets/RPDC188/val_1000: 71.52%
  [Seed 101] ../datasets/RPDC197/train_50 → ../datasets/RPDC191/val_1000: 80.52%
  [Seed 101] ../datasets/RPDC197/train_50 → ../datasets/RPDC194/val_1000: 82.80%
  [Seed 101] ../datasets/RPDC197/train_50 → ../datasets/RPDC197/val_1000: 94.90%
  [Seed 101] ../datasets/RPDC197/train_100 → ../datasets/RPDC185/val_1000: 75.55%
  [Seed 101] ../datasets/RPDC197/train_100 → ../datasets/RPDC188/val_1000: 

In [ ]:
=== Mean ± Std Dev over seeds ===
../datasets/RPDC197/train_20 → ../datasets/RPDC185/val_1000: mean = 65.49%,  std = 3.49%
../datasets/RPDC197/train_20 → ../datasets/RPDC188/val_1000: mean = 62.26%,  std = 5.24%
../datasets/RPDC197/train_20 → ../datasets/RPDC191/val_1000: mean = 83.42%,  std = 2.76%
../datasets/RPDC197/train_20 → ../datasets/RPDC194/val_1000: mean = 87.57%,  std = 4.30%
../datasets/RPDC197/train_20 → ../datasets/RPDC197/val_1000: mean = 94.84%,  std = 1.97%
../datasets/RPDC197/train_50 → ../datasets/RPDC185/val_1000: mean = 68.85%,  std = 6.64%
../datasets/RPDC197/train_50 → ../datasets/RPDC188/val_1000: mean = 72.06%,  std = 5.86%
../datasets/RPDC197/train_50 → ../datasets/RPDC191/val_1000: mean = 86.13%,  std = 3.81%
../datasets/RPDC197/train_50 → ../datasets/RPDC194/val_1000: mean = 89.62%,  std = 4.92%
../datasets/RPDC197/train_50 → ../datasets/RPDC197/val_1000: mean = 95.91%,  std = 1.13%
../datasets/RPDC197/train_100 → ../datasets/RPDC185/val_1000: mean = 77.05%,  std = 2.62%
../datasets/RPDC197/train_100 → ../datasets/RPDC188/val_1000: mean = 79.96%,  std = 3.75%
../datasets/RPDC197/train_100 → ../datasets/RPDC191/val_1000: mean = 90.50%,  std = 2.15%
../datasets/RPDC197/train_100 → ../datasets/RPDC194/val_1000: mean = 94.82%,  std = 3.65%
../datasets/RPDC197/train_100 → ../datasets/RPDC197/val_1000: mean = 98.13%,  std = 0.33%
../datasets/RPDC197/train_200 → ../datasets/RPDC185/val_1000: mean = 78.39%,  std = 2.04%
../datasets/RPDC197/train_200 → ../datasets/RPDC188/val_1000: mean = 82.46%,  std = 3.94%
../datasets/RPDC197/train_200 → ../datasets/RPDC191/val_1000: mean = 91.96%,  std = 2.34%
../datasets/RPDC197/train_200 → ../datasets/RPDC194/val_1000: mean = 94.91%,  std = 3.52%
../datasets/RPDC197/train_200 → ../datasets/RPDC197/val_1000: mean = 98.67%,  std = 0.39%
../datasets/RPDC197/train_300 → ../datasets/RPDC185/val_1000: mean = 77.07%,  std = 1.93%
../datasets/RPDC197/train_300 → ../datasets/RPDC188/val_1000: mean = 80.61%,  std = 2.84%
../datasets/RPDC197/train_300 → ../datasets/RPDC191/val_1000: mean = 92.70%,  std = 2.23%
../datasets/RPDC197/train_300 → ../datasets/RPDC194/val_1000: mean = 93.58%,  std = 4.60%
../datasets/RPDC197/train_300 → ../datasets/RPDC197/val_1000: mean = 99.49%,  std = 0.33%
../datasets/RPDC197/train_400 → ../datasets/RPDC185/val_1000: mean = 75.88%,  std = 2.49%
../datasets/RPDC197/train_400 → ../datasets/RPDC188/val_1000: mean = 77.66%,  std = 4.03%
../datasets/RPDC197/train_400 → ../datasets/RPDC191/val_1000: mean = 92.96%,  std = 1.85%
../datasets/RPDC197/train_400 → ../datasets/RPDC194/val_1000: mean = 93.52%,  std = 3.91%
../datasets/RPDC197/train_400 → ../datasets/RPDC197/val_1000: mean = 99.45%,  std = 0.29%
../datasets/RPDC197/train_500 → ../datasets/RPDC185/val_1000: mean = 76.62%,  std = 2.91%
../datasets/RPDC197/train_500 → ../datasets/RPDC188/val_1000: mean = 79.40%,  std = 5.17%
../datasets/RPDC197/train_500 → ../datasets/RPDC191/val_1000: mean = 93.02%,  std = 1.90%
../datasets/RPDC197/train_500 → ../datasets/RPDC194/val_1000: mean = 93.48%,  std = 4.69%
../datasets/RPDC197/train_500 → ../datasets/RPDC197/val_1000: mean = 99.57%,  std = 0.25%
../datasets/RPDC197/train_600 → ../datasets/RPDC185/val_1000: mean = 75.90%,  std = 3.59%
../datasets/RPDC197/train_600 → ../datasets/RPDC188/val_1000: mean = 79.44%,  std = 4.63%
../datasets/RPDC197/train_600 → ../datasets/RPDC191/val_1000: mean = 93.25%,  std = 1.99%
../datasets/RPDC197/train_600 → ../datasets/RPDC194/val_1000: mean = 92.97%,  std = 5.75%
../datasets/RPDC197/train_600 → ../datasets/RPDC197/val_1000: mean = 99.62%,  std = 0.15%